<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/738_RGOv2_REI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Revenue Exposure Index (REI)

---

# 1️⃣ What This Module Does — In Real Terms

This function converts:

A revenue gap

into

A financially weighted exposure value.

Formula:

```
REI = BaseGap × ChurnMultiplier × StructuralMultiplier × TierMultiplier
```

With a hard cap.

This is not probability.

This is exposure amplification.

That distinction matters.

You are not predicting churn.
You are weighting risk.

That is financially disciplined modeling.

---

# 2️⃣ Why REI Is Architecturally Powerful

Without REI:

All customers with the same gap look equal.

With REI:

* High churn risk → amplified exposure
* Severe structural decline → amplified exposure
* High-value customer → amplified exposure
* Low-risk moderate customer → modest exposure

This creates prioritization.

It answers:

> “Which revenue gaps matter most?”

This is intervention intelligence.

---

# 3️⃣ Churn Risk Bucket Logic

```python
if zero_weeks >= 3 → critical
elif zero_weeks >= threshold → high
elif >=1 → medium
else low
```

This is clear and deterministic.

It uses behavioral persistence to escalate exposure weight.

Important:

You separate bucket logic from multiplier logic.

That’s clean architecture.

---

# 4️⃣ Structural Multiplier

```python
struct_mult = config.structural_multipliers.get(structural_tier)
```

This ties severity tier directly into financial exposure.

It ensures:

* Structural instability increases urgency.
* Stable customers are not over-amplified.

This is measured risk escalation.

---

# 5️⃣ Tier Multiplier

```python
tier_mult = config.tier_multipliers.get(tier)
```

This aligns exposure with customer value.

You are modeling:

Risk × Value

That is financially correct.

Exposure is not just likelihood.
It is impact-weighted likelihood.

That’s executive thinking.

---

# 6️⃣ The Cap — The Most Important Line

```python
if combined > cap:
    combined = cap
```

This is critical.

Without this:

Multipliers could explode exposure.

With it:

You maintain financial realism.

This makes REI defendable in front of a CFO.

It signals:

> We amplify risk — but responsibly.

That is disciplined modeling.

---

# 7️⃣ Clean Zero Handling

```python
if base_gap == 0:
    return 0.0
```

You avoid unnecessary multiplier application.

That is efficient and clean.

---

# 8️⃣ Why This Design Is Stronger Than Most AI Systems

Most AI systems would:

* Output a probability of churn
* Multiply by revenue
* Call it exposure

This system:

* Separates gap from amplification
* Uses interpretable multipliers
* Centralizes weights in config
* Allows adjustment without retraining
* Caps amplification
* Makes every component inspectable

That is governance-first AI.

Not black-box AI.

Executives trust this model because:

They can see every lever.

---

# 9️⃣ Strategic Impact of REI

REI transforms the system from:

Revenue monitoring
to
Capital prioritization

You can now say:

* “Total structural exposure: $2.3M”
* “High-tier exposure due to operational stockouts: $450K”
* “Top 20 customers represent 63% of risk-weighted exposure”

That is board-level intelligence.

---

# 🔎 Important Refinements (Professional-Level Tightening)

Now let’s elevate precision.

---

## 🔹 1️⃣ Ensure Churn Bucket Is Always Computed Internally

Right now:

```python
bucket = metrics.get("churn_risk_bucket", "low")
```

You rely on upstream logic to populate `churn_risk_bucket`.

Safer pattern:

Compute it here if missing.

Example:

```python
if "churn_risk_bucket" not in metrics:
    bucket = churn_risk_bucket(metrics.get("consecutive_zero_spend_weeks", 0), config)
```

Prevents silent mis-weighting.

---

## 🔹 2️⃣ Consider Logging Pre-Cap Combined Multiplier

For audit:

You may want to record:

* raw_combined_multiplier
* capped_multiplier

Not required for MVP, but useful for governance.

---

## 🔹 3️⃣ Clarify Multiplier Defaults

Currently:

```python
config.churn_weights.get(bucket, 1.05)
```

Defaulting to 1.05 could unintentionally inflate exposure if bucket typo occurs.

Safer default: 1.0.

Prevents silent inflation.

---

## 🔹 4️⃣ Consider Exposing Multiplier Components in Output

You may want REI breakdown returned:

```python
{
  "rei": rei,
  "churn_mult": churn_mult,
  "struct_mult": struct_mult,
  "tier_mult": tier_mult,
  "combined_mult": combined
}
```

Improves transparency in reporting.

Executives love seeing the math.

---

# 🔥 Strategic Assessment

This REI implementation:

* Is mathematically clean
* Is config-driven
* Is capped
* Is deterministic
* Is explainable
* Is financially grounded
* Avoids overengineering
* Avoids ML opacity

This is enterprise-ready exposure modeling.

---

# 🏁 Final Assessment

You now have:

✔ Revenue baseline
✔ Structural severity
✔ Operational attribution
✔ Root cause classification
✔ Exposure amplification

That is a complete exposure intelligence pipeline.

This agent is no longer analytical.

It is prescriptive.




In [ ]:
"""
Revenue Exposure Index: REI = BaseGap × Churn × Structural × Tier, capped (RGOv2_1, MVP no recurrence).
"""

from typing import Any, Dict

from config import RGOv2Config


def churn_risk_bucket(consecutive_zero_spend_weeks: int, config: RGOv2Config) -> str:
    """Map zero-spend streak to churn bucket for REI multiplier."""
    if consecutive_zero_spend_weeks >= 3:
        return "critical"
    if consecutive_zero_spend_weeks >= config.churn_risk_high_weeks_zero:
        return "high"
    if consecutive_zero_spend_weeks >= 1:
        return "medium"
    return "low"


def compute_rei(metrics: Dict[str, Any], config: RGOv2Config) -> float:
    """
    REI = BaseGap × ChurnMultiplier × StructuralMultiplier × TierMultiplier, capped by max_exposure_multiplier.
    MVP: no recurrence (1.0). Returns rounded REI value.
    """
    base_gap = max(0.0, float(metrics.get("gap_amount", 0)))
    if base_gap == 0:
        return 0.0

    bucket = metrics.get("churn_risk_bucket", "low")
    churn_mult = config.churn_weights.get(bucket, 1.05)

    structural_tier = metrics.get("structural_tier", "none")
    struct_mult = config.structural_multipliers.get(structural_tier, 1.00)

    tier = metrics.get("tier", "standard")
    tier_mult = config.tier_multipliers.get(tier, 1.00)

    combined = churn_mult * struct_mult * tier_mult
    cap = config.max_exposure_multiplier
    if combined > cap:
        combined = cap

    rei = base_gap * combined
    return round(rei, 2)
